# Contenidos <a id='back'></a>

* [Etapa 1. Introducción](#intro)
    * [1.1 Objetivo](#objetivo)
    * [1.2 Etapas](#etapas)
    * [1.3 Archivo](#arch)
* [Etapa 2. Exploración de Datos](#data_preprocessing)
    * [2.1 Descripción de los datos](#descripción)
    * [2.2 Valores ausentes](#missing_values)
    * [2.3 Conclusiones intermedias](#int_con)
    * [2.4 Conclusiones](#data_preprocessing_conclusions)
* [Etapa 3. Transformación de datos](#data_transformation)
    * [3.1 Restaurar valores ausentes en 'total_income'](#restore_total_income)
    * [3.2 Restaurar valores en 'days_employed'](#restore_days_employed)
* [Etapa 4. Clasificación de datos](#data_classification)
* [Etapa 5. Prueba de hipótesis](#hypotheses)
    * [3.1 Hipótesis 1: Comprobar si existe una correlación entre tener hijos y pagar a tiempo](#kids)
    * [3.2 Hipótesis 2: Analizar si existe una correlación entre la situación familiar y el pago a tiempo](#family)
    * [3.3 Hipótesis 3: Probar si existe una correlación entre el nivel de ingresos y el pago a tiempo](#income)
    * [3.4 Hopótesis 4: Investigar cómo afecta el propósito del crédito a la tasa de incumplimiento](#purpose)
* [Conclusiones](#end)

## Introducción <a id='intro'></a>
En este proyecto se realizará un análisis del riesgo de incumplimiento de los prestatarios y consiste en preparar un informe para la división de préstamos de un banco. Se deberá averiguar si el estado civil y el número de hijos de un cliente tienen un impacto en el incumplimiento de pago de un préstamo. El banco ya tiene algunos datos sobre la solvencia crediticia de los clientes.

Este informe se tendrá en cuenta al crear una puntuación de crédito para un cliente potencial. La puntuación de crédito se utiliza para evaluar la capacidad de un prestatario potencial para pagar su préstamo.

### Objetivo <a id='objetivo'></a>

1. Comprobar si existe una correlación entre tener hijos y pagar a tiempo.
2. Analizar si existe una correlación entre la situación familiar y el pago a tiempo.
3. Probar si existe una correlación entre el nivel de ingresos y el pago a tiempo.

### Etapas <a id='etapas'></a>
1. Introducción
2. Exploración de datos
3. Transformación de datos
4. Clasificación de datos
5. Prueba de hipótesis

### Abrir el archivo de datos y mirar la información general <a id='arch'></a>

In [95]:
import pandas as pd
import math

In [96]:
credit = pd.read_csv(r'G:\datasets\credit_scoring_eng.csv')

## Exploración de datos <a id='data_preprocessing'></a>

### Descripción de los datos <a id='descripción'></a>
- `children` - el número de hijos en la familia
- `days_employed` - experiencia laboral en días
- `dob_years` - la edad del cliente en años
- `education` - la educación del cliente
- `education_id` - identificador de educación
- `family_status` - estado civil
- `family_status_id` - identificador de estado civil
- `gender` - género del cliente
- `income_type` - tipo de empleo
- `debt` - ¿había alguna deuda en el pago de un préstamo?
- `total_income` - ingreso mensual
- `purpose` - el propósito de obtener un préstamo

In [97]:
credit.shape

(21525, 12)

In [98]:
credit.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


La columna de `days_employed` muesta:
- Números negativos.
- Cantidad de días equivocados. Por ejemplo, la fila con el index 4 muestra 340266.07 días con empleo, esto son 932+ años.

La columna `education` contiene valores con diferentes formatos, mayúsculas y minúsculas.

La columna `purpose` tiene el peligro de contener el mismo valor escritos de diferente manera por ejemplo 'car purchase' y simplemente 'car' o 'cars'.

In [99]:
credit.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Hay valores ausentes en las columnas days_employed y total_income. Los tipos de datos parecen estar en orden.

### Valores ausentes <a id='missing_values'></a>

In [100]:
credit.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Los valores ausentes parecen ser simétricos ya que solo son dos columnas a las cuales les faltan datos. Es probable que se encuentra una correlación entre ambas. Se requiere realizar más investigación para poder determinar si todos los datos faltantes son de las mismas filas.

In [101]:
missing = credit.loc[credit['days_employed'].isna(), 'total_income'].eq('NaN')
print(missing)
missing.value_counts()

12       False
26       False
29       False
41       False
55       False
         ...  
21489    False
21495    False
21497    False
21502    False
21510    False
Name: total_income, Length: 2174, dtype: bool


False    2174
Name: total_income, dtype: int64

### Conclusión intermedia <a id='int_con'></a>

Realizando la tabla filtrada nos podemos dar cuenta de que en efecto, todos los datos ausentes en `days_employed` son los mismos ausentes en `total_income`, la tabla anterior muestra esta afirmación dando como resultado las filas en las que en ambas se tienen valores ausentes simultaneamente, regresando `False` si así es. Como se puede notar, gracias al uso del método `.value_counts()`, nos regresa tan solo un resultado, `False`, con 2,174 apariciones las cuales también coinciden con la cantidad de datos ausentes totales (19,351 + 2,174 = 21,525).

Ahora bien, se calculará el porcentaje de los valores ausentes en comparación con el conjunto de datos completo para determinar si se trata de una porción de datos considerablemenre grande, de ser así, se completrarán los valores ausentes. Para lograrlo, se tomarán en cuenta las columnas con valores ausentes y el `income_type` para observar si la mayoría de estos datos ausentes se deben a que la persona está retirada u otro factor a considerar.

Se calcularán los datos duplicados para ver si afectan a los datos totales.

In [102]:
#Cuantos datos son duplicados
duplicates = credit[credit.duplicated()]
print(duplicates.count())
print()

#Veamos cuanto porcentaje de datos está ausente
print(math.ceil((missing.count() / credit['children'].count()) * 100),'%')

children            54
days_employed        0
dob_years           54
education           54
education_id        54
family_status       54
family_status_id    54
gender              54
income_type         54
debt                54
total_income         0
purpose             54
dtype: int64

11 %


In [103]:
#Asegurando que el income_type no sea un factor para los datos ausentes
credit[credit['days_employed'].isna()].pivot_table(index='income_type',values='total_income' ,aggfunc='count')

,total_income
income_type,
business,0
civil servant,0
employee,0
entrepreneur,0
retiree,0


In [104]:
#Calculando la media de total_income por income_type
credit.groupby('income_type').agg({'total_income': 'mean'}).reset_index()

,income_type,total_income
0,business,32386.793835
1,civil servant,27343.729582
2,employee,25820.841683
3,entrepreneur,79866.103000
4,paternity / maternity leave,8612.661000
5,retiree,21940.394503
6,student,15712.260000
7,unemployed,21014.360500


Resulta ser que los datos ausentes representan el 10% de nuestros datos. Debido a que el 10% de los datos es una cantidad muy grande, se tendrá que hacer algo al respecto. 

Es posible que estos datos se hayan perdido debido al formato que tienen ya que de igual manera es incorrecto, es decir, la columna de days_employed debe tener un error muy grande con los días y/o fechas que se trabajaron. Es por esto, que el mejor acercamiento es sustituir los datos ausentes con la mediana.

El DataFrame cuenta con 54 datos duplicados. 

In [105]:
# Comprobando la distribución en el conjunto de datos entero
for col in credit.columns:
    print(credit[col].value_counts(normalize = True))

 0     0.657329
 1     0.223833
 2     0.095470
 3     0.015331
 20    0.003531
-1     0.002184
 4     0.001905
 5     0.000418
Name: children, dtype: float64
-8437.673028      0.000052
-3507.818775      0.000052
 354500.415854    0.000052
-769.717438       0.000052
-3963.590317      0.000052
                    ...   
-1099.957609      0.000052
-209.984794       0.000052
 398099.392433    0.000052
-1271.038880      0.000052
-1984.507589      0.000052
Name: days_employed, Length: 19351, dtype: float64
35    0.028664
40    0.028293
41    0.028200
34    0.028014
38    0.027782
42    0.027735
33    0.026992
39    0.026620
31    0.026016
36    0.025784
44    0.025412
29    0.025319
30    0.025087
48    0.024994
37    0.024948
50    0.023879
43    0.023833
32    0.023693
49    0.023600
28    0.023368
45    0.023089
27    0.022904
56    0.022625
52    0.022485
47    0.022300
54    0.022253
46    0.022067
58    0.021417
57    0.021370
53    0.021324
51    0.020813
59    0.020627
55    0.02058

In [106]:
for col in credit.columns:
    print(credit.dropna()[col].value_counts(normalize = True))

 0     0.656814
 1     0.224433
 2     0.095654
 3     0.015193
 20    0.003462
-1     0.002274
 4     0.001757
 5     0.000413
Name: children, dtype: float64
-8437.673028      0.000052
-3507.818775      0.000052
 354500.415854    0.000052
-769.717438       0.000052
-3963.590317      0.000052
                    ...   
-1099.957609      0.000052
-209.984794       0.000052
 398099.392433    0.000052
-1271.038880      0.000052
-1984.507589      0.000052
Name: days_employed, Length: 19351, dtype: float64
35    0.028577
41    0.028319
38    0.028112
40    0.028061
34    0.027595
42    0.027492
33    0.027389
39    0.026975
44    0.025993
29    0.025580
31    0.025580
48    0.025425
36    0.025425
37    0.025012
30    0.024908
32    0.024443
43    0.023926
50    0.023926
49    0.023668
27    0.023616
45    0.023100
28    0.023048
56    0.022376
52    0.022273
46    0.022066
54    0.021911
47    0.021756
53    0.021446
59    0.021188
58    0.020929
57    0.020877
51    0.020567
55    0.02041

**Conclusión intermedia**

Como se puede notar, el análisis anterior mostró que no hay un cambio importante en los porcentajes de los datos ausentes en las demás columnas. Debido a esto, lo más probable es que los datos ausentes esten presentes de manera accidental y aleatoria.

A continuación, se sigue llevando a cabo un análisis filtrando las columnas con mayor claridad.

In [107]:
credit_missing = credit[['children', 'education', 'education_id', 'family_status', 'family_status_id', 'gender', 'income_type', 'debt', 'purpose']]
for col in credit_missing.columns:
    print(credit[col].value_counts(normalize = True))

 0     0.657329
 1     0.223833
 2     0.095470
 3     0.015331
 20    0.003531
-1     0.002184
 4     0.001905
 5     0.000418
Name: children, dtype: float64
secondary education    0.638792
bachelor's degree      0.219187
SECONDARY EDUCATION    0.035865
Secondary Education    0.033031
some college           0.031034
BACHELOR'S DEGREE      0.012729
Bachelor's Degree      0.012451
primary education      0.011614
Some College           0.002184
SOME COLLEGE           0.001347
PRIMARY EDUCATION      0.000790
Primary Education      0.000697
graduate degree        0.000186
Graduate Degree        0.000046
GRADUATE DEGREE        0.000046
Name: education, dtype: float64
1    0.707689
0    0.244367
2    0.034564
3    0.013101
4    0.000279
Name: education_id, dtype: float64
married              0.575145
civil partnership    0.194053
unmarried            0.130685
divorced             0.055517
widow / widower      0.044599
Name: family_status, dtype: float64
0    0.575145
1    0.194053
4    0.130

In [108]:
for col in credit_missing.columns:
    print(credit.dropna()[col].value_counts(normalize = True))

 0     0.656814
 1     0.224433
 2     0.095654
 3     0.015193
 20    0.003462
-1     0.002274
 4     0.001757
 5     0.000413
Name: children, dtype: float64
secondary education    0.637796
bachelor's degree      0.218180
SECONDARY EDUCATION    0.036432
Secondary Education    0.033383
some college           0.031678
BACHELOR'S DEGREE      0.012971
Bachelor's Degree      0.012557
primary education      0.011937
Some College           0.002067
SOME COLLEGE           0.001137
PRIMARY EDUCATION      0.000827
Primary Education      0.000723
graduate degree        0.000207
Graduate Degree        0.000052
GRADUATE DEGREE        0.000052
Name: education, dtype: float64
1    0.707612
0    0.243708
2    0.034882
3    0.013488
4    0.000310
Name: education_id, dtype: float64
married              0.575836
civil partnership    0.193013
unmarried            0.130484
divorced             0.055966
widow / widower      0.044701
Name: family_status, dtype: float64
0    0.575836
1    0.193013
4    0.130

**Conclusión intermedia**

Asegurando que los cambios en los porcentajes son mínimos, podemos concluir que en efecto, los valores ausentes son accidentales y aleatorios.

### Conclusiones <a id='data_preprocessing_conclusions'></a>

Debido a que no se encontraron patrones en el cambio de porcentajes en los datos, los valores ausentes accidentales y aleatorios.

Para abordar los valores ausente se tomará en cuenta el contexto de estos y se determinará la mejor solución. Ya que los datos ausentes son numéricos, este problema se resolverá utilizando mediana.

En la siguiente etapa se tomarán en cuenta para la tranformación y limpieza del DataFrame los valores duplicados, diferentes registros, artefactos incorrectos y valores ausentes. En cada uno de estos problemas se le dará un seguimiento adecuado.

## Transformación de datos <a id='data_transformation'></a>

De simple vista, se puede observar que la columna `education` tiene datos con el mismo valor que han sido escritos de diferente manera, ya sea con una mayúscula, toda la oración en mayúsculas, etc.

Algo similar ocurre en la columna `purpose`.

In [109]:
# Veamos todos los valores en la columna de educación para verificar si será necesario corregir la ortografía y qué habrá que corregir exactamente
credit['education'].head(10)

0      bachelor's degree
1    secondary education
2    Secondary Education
3    secondary education
4    secondary education
5      bachelor's degree
6      bachelor's degree
7    SECONDARY EDUCATION
8      BACHELOR'S DEGREE
9    secondary education
Name: education, dtype: object

In [110]:
credit['education'] = credit['education'].str.lower()

In [111]:
credit['education']

0          bachelor's degree
1        secondary education
2        secondary education
3        secondary education
4        secondary education
                ...         
21520    secondary education
21521    secondary education
21522    secondary education
21523    secondary education
21524    secondary education
Name: education, Length: 21525, dtype: object

In [112]:
# Veamos la distribución de los valores en la columna `children`
credit['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Se puede notar que existen dos conjuntos de datos en la columna `children` que es muy probable que tengan error, el de -1 y el de 20 ya que no se puede tener -1 hijo y tener 20 hijos es algo impensable además, los datos muestran que entre más alta sea la cantidad de hijos, menos casos hay, por ejemplo, se encuentran 2,055 personas con 2 hijos y tan solo 9 con 5. Para solucionar este problema, primero se calculará la media de los datos para determinar si es viable utilizarla para llenar los datos equivocados. De no serlo, se corregirá los dígitos con problemas para cambiar el 20 por un 2 y el -1 por un 1 ya que lo más probable es que estos datos hayan sido escritos erroneamente por un humano.

In [113]:
credit['children'].mean()

0.5389082462253194

Ya que no es posible tener 0.54 hijos, se redondearán hacia arriba. Teniendo esto en cuenta, todos los datos con problema serían transfomrados a 1. Se ha considerado que un mejor acercamiento para solucionar este error es sustituir los datos con errores de 20 y -1 por 2 y 1, respectivamente.

In [114]:
credit.head(10)
credit['children'] = credit['children'].replace([-1, 20], [1, 2])

In [115]:
credit['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

Desde un principio se notó que la columna de `days_employed` tenía un problema crítico para el análisis correcto de los datos, muchos están en negativos y otros tienen cantidades imposibles de días trabajados. A continuación se calculará el porcentaje de datos problemáticos en esta columna.

Para determinar si la cantidad de días es correcto o no, se tomarán en cuenta los trabajadores más jóvenes (19 años) y los más ancianos (75 años), esto nos da un rango de 56 años (75 - 19)(20,440 días) trabajados máximo  y será el dato que se utilizará para el análisis de la columna.

In [116]:
credit[credit['days_employed'] > 0].value_counts('income_type')

income_type
retiree       3443
unemployed       2
dtype: int64

In [117]:
credit[credit['days_employed'] <= 20440].value_counts('income_type')

income_type
employee                       10014
business                        4577
civil servant                   1312
entrepreneur                       1
paternity / maternity leave        1
student                            1
dtype: int64

In [118]:
print(sum(1 for i in credit['days_employed'] if i > 0))
print(sum(1 for i in credit['days_employed'] if i <= 20440))
(credit[(credit['days_employed'] > 0) & (credit['days_employed'] <= 20440)])

3445
15906


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


In [119]:
credit[credit['income_type'].str.contains('retiree') & (credit['dob_years'] != 0) & (credit['dob_years'] < 35)]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
1242,0,334764.259831,22,secondary education,1,unmarried,4,F,retiree,0,14298.976,getting higher education
3619,0,NaN,24,secondary education,1,married,0,F,retiree,0,NaN,purchase of my own house
3963,0,391961.274017,34,secondary education,1,unmarried,4,M,retiree,0,12785.054,university education
5717,0,397905.003200,33,secondary education,1,civil partnership,1,M,retiree,0,15083.208,having a wedding
5842,2,397548.767244,34,secondary education,1,married,0,F,retiree,0,47686.626,transactions with commercial real estate
...,...,...,...,...,...,...,...,...,...,...,...,...
18175,2,NaN,31,secondary education,1,civil partnership,1,F,retiree,0,NaN,wedding ceremony
18699,0,368556.010382,34,secondary education,1,civil partnership,1,M,retiree,0,11795.272,to have a wedding
19417,1,350340.760224,28,secondary education,1,divorced,3,F,retiree,0,8459.679,cars
19439,0,389397.167577,26,bachelor's degree,0,married,0,F,retiree,0,34394.128,buy real estate


In [120]:
years_employed = credit['days_employed'] / 365

Después de realizar el calculo se puede notar que absolutamente todos los datos muestran problema, ya sea que tienen un número negativo o son una cantidad de años imposibles de alcanzar. 

Es probable que este error haya sido causado por un problema técnico ya que más del 99% de los datos con una cantidad de años trabajados abrumadora pertenecen a personas con un `income_type` de retirado y solo dos filas tienen uno de unemployed. Esto pudo haber sido ocasionado por una falla en el calculo ya que en el dataframe se cuenta por ejemplo, con personas con 22 años que han trabajado durante 917 años. Aún no es posible rellenar los valores ausentes ya que ni la media, ni la mediana ni moda serían útiles hasta que este error sea solucionado. Para solucionar esta problemática se dividió la cantidad de días para transformarlos a años y apartir de eso se puede encontrar un patrón que demuestra -----.

En cuanto a los datos negativos, es también muy probable que haya sido un error técnico y simplemente se deba de eliminar el signo negativo para solucionar el problema.

In [121]:
credit['days_employed'] = credit['days_employed'].abs()
years_employed = years_employed.abs()

In [122]:
years_employed.head(15)

0      23.116912
1      11.026860
2      15.406637
3      11.300677
4     932.235814
5       2.537495
6       7.888225
7       0.418574
8      18.985932
9       5.996593
10     11.428722
11      2.171786
12           NaN
13      5.059293
14      5.054674
Name: days_employed, dtype: float64

Se analizará si se cuenta con alguna edad de cliente que no sea posible, de ser cierto, se corregirá

In [123]:
print(credit['dob_years'].value_counts())
print((sum(credit.dob_years == 0) / sum(credit.dob_years)* 100),'%')

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
22    183
66    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64
0.010838189056648316 %


Se encontró un dato con valor de 0 y hay 101 entradas con este valor los cuales son equivalente al 0.01% del total de los datos. Para solucionarlo, se reemplazaran por la media de los datos ya que no se encuentran datos críticos.

In [124]:
credit['dob_years'] = credit['dob_years'].replace(0, int(credit['dob_years'].mean()))


In [125]:
print(credit['dob_years'].value_counts())


35    617
43    614
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64


Ahora revisemos la columna `family_status`

In [126]:
print(credit['family_status'].value_counts(), credit['family_status_id'].value_counts())
print()
credit['family_status'].isna().count()

married              12380
civil partnership     4177
unmarried             2813
divorced              1195
widow / widower        960
Name: family_status, dtype: int64 0    12380
1     4177
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64



21525

In [127]:
credit.loc[:, ['family_status', 'family_status_id']].value_counts()

family_status      family_status_id
married            0                   12380
civil partnership  1                    4177
unmarried          4                    2813
divorced           3                    1195
widow / widower    2                     960
dtype: int64

Las columnas `family_status` y `family_status_id` parecen estar libres de problemas.

En el caso de la columna `gender` se tendrá que tomar en cuenta que si hay más de dos variables, se transformarán esos datos utilizando la moda para determinar su valor final.

In [128]:
credit['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [129]:
credit['gender'].mode()

0    F
dtype: object

In [130]:
credit['gender'] = credit['gender'].replace('XNA', 'F')

In [131]:
credit['gender'].value_counts()

F    14237
M     7288
Name: gender, dtype: int64

Ahora se revisará la columna `income_type`.

In [132]:
print(credit['income_type'].value_counts())
print()
credit['income_type'].isna().count()

employee                       11119
business                        5085
retiree                         3856
civil servant                   1459
unemployed                         2
entrepreneur                       2
student                            1
paternity / maternity leave        1
Name: income_type, dtype: int64



21525

No se encontraron errores en `income_type`

Por la naturaleza de la tabla y los datos en ella, habrá miles de duplicados sin embargo, eso no significa que deban de ser cambiados o borrados ya que habrá muchas personas con el dato `married`, con un hijo y que tengan 50 años, por ejemplo:

In [133]:
credit[credit['family_status'].str.contains('married') & (credit['dob_years'] == 50) & (credit['children'] == 1)]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
116,1,540.038425,50,secondary education,1,married,0,M,employee,0,13338.611,buy residential real estate
233,1,1475.697490,50,secondary education,1,married,0,F,employee,0,22942.571,to buy a car
570,1,3806.562913,50,secondary education,1,married,0,M,employee,0,23409.661,building a real estate
705,1,902.084528,50,secondary education,1,married,0,F,civil servant,0,22061.264,car purchase
877,1,558.262799,50,secondary education,1,married,0,M,business,0,36210.807,housing renovation
...,...,...,...,...,...,...,...,...,...,...,...,...
18669,1,1953.777603,50,secondary education,1,married,0,F,civil servant,0,4672.012,cars
19661,1,335525.912971,50,secondary education,1,married,0,F,retiree,0,25147.636,buy residential real estate
19679,1,3924.380059,50,secondary education,1,married,0,M,civil servant,0,16681.436,property
20463,1,3376.486227,50,secondary education,1,married,0,F,civil servant,0,24139.713,purchase of my own house


En el ejemplo anterior se muestra una de las muchas casualidades y "duplicados" que puede llegar a haber en la tabla. Los datos más importantes para determinar si existen realmente duplicados son las columnas `days_employed` y `total_income`.

In [134]:
credit_size = credit.groupby(credit.columns.tolist(), as_index = False).size()
credit_size

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,size
0,0,24.240695,32,bachelor's degree,0,unmarried,4,M,employee,0,19858.460,to get a supplementary education,1
1,0,33.520665,43,secondary education,1,unmarried,4,M,employee,1,20568.944,car purchase,1
2,0,39.954170,34,bachelor's degree,0,married,0,F,civil servant,0,15562.363,housing,1
3,0,47.109840,49,bachelor's degree,0,married,0,F,employee,0,31607.243,buying a second-hand car,1
4,0,50.128298,43,secondary education,1,civil partnership,1,F,business,0,15901.112,having a wedding,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19346,5,773.124856,36,secondary education,1,married,0,F,employee,0,7803.663,housing transactions,1
19347,5,1690.018117,59,secondary education,1,married,0,M,employee,0,43050.936,transactions with my real estate,1
19348,5,2286.262752,37,secondary education,1,married,0,F,employee,0,41071.736,buy real estate,1
19349,5,2386.600221,35,secondary education,1,married,0,F,business,0,32678.703,housing,1


In [135]:
print(credit_size['size'].value_counts())
print()
credit_size.count()

1    19351
Name: size, dtype: int64



children            19351
days_employed       19351
dob_years           19351
education           19351
education_id        19351
family_status       19351
family_status_id    19351
gender              19351
income_type         19351
debt                19351
total_income        19351
purpose             19351
size                19351
dtype: int64

In [136]:
credit[credit.duplicated()].sort_values(by = ['dob_years', 'education', 'family_status'])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
20297,1,NaN,23,secondary education,1,civil partnership,1,F,employee,0,NaN,to have a wedding
19321,0,NaN,23,secondary education,1,unmarried,4,F,employee,0,NaN,second-hand car purchase
18328,0,NaN,29,bachelor's degree,0,married,0,M,employee,0,NaN,buy residential real estate
18349,1,NaN,30,bachelor's degree,0,married,0,F,civil servant,0,NaN,purchase of the house for my family
21281,1,NaN,30,bachelor's degree,0,married,0,F,employee,0,NaN,buy commercial real estate
...,...,...,...,...,...,...,...,...,...,...,...,...
20702,0,NaN,64,secondary education,1,married,0,F,retiree,0,NaN,supplementary education
20187,0,NaN,65,secondary education,1,civil partnership,1,F,retiree,0,NaN,to have a wedding
9528,0,NaN,66,secondary education,1,widow / widower,2,F,retiree,0,NaN,transactions with my real estate
7938,0,NaN,71,secondary education,1,civil partnership,1,F,retiree,0,NaN,having a wedding


In [137]:
credit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [138]:
credit.duplicated().sum()/len(credit)*100

0.32984901277584205

In [139]:
credit.duplicated().sum()

71

In [140]:
credit = credit.drop_duplicates().reset_index(drop = True)
credit.shape

(21454, 12)

Se comprobró que el DataFrame contaba con 71 datos duplicados los cuales representan el 0.33% de los datos, por esto, se tomó la decisión de utilizar `drop_duplicates` para limpiar los datos

### Restaurar valores ausentes en `total_income` <a id='restore_total_income'></a>

Como ha sido previamente mencionado, las columnas `total_income` y `days_employed` tienen valores ausentes.

In [141]:
credit[credit['total_income'].isnull()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21418,2,NaN,47,secondary education,1,married,0,M,business,0,NaN,purchase of a car
21424,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21426,0,NaN,48,bachelor's degree,0,married,0,F,business,0,NaN,building a property
21431,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


Se comenzará por analizar las columnas con datos más relevantes para este caso. Se utilizará `median` debido a que existen valores atipicos.

In [142]:
education_median = credit.groupby('education').agg({'total_income': 'median'}).reset_index()
education_median

,education,total_income
0,bachelor's degree,28054.5310
1,graduate degree,25161.5835
2,primary education,18741.9760
3,secondary education,21836.5830
4,some college,25618.4640


Es importante notar que en efecto, entre más son los estudios de las personas, más alto es el ingreso total, esto puede ser relevante más adelante. De la misma manera, se analizará si la edad es un factor importante.

In [143]:
dob_years_median = credit.groupby('dob_years').agg({'total_income': 'median'}).reset_index()
print(dob_years_median.head(10))
dob_years_median.tail(10)

   dob_years  total_income
0         19    14934.9010
1         20    17520.3290
2         21    20522.5150
3         22    19839.3410
4         23    19706.0455
5         24    22464.4030
6         25    22899.0990
7         26    23273.1360
8         27    24708.6790
9         28    23946.0940


,dob_years,total_income
47,66,17867.7550
48,67,18024.9680
49,68,18113.3420
50,69,16690.0775
51,70,18889.9720
52,71,19669.4870
53,72,17757.6840
54,73,19070.4780
55,74,12296.0520
56,75,24525.2240


Esta métrica no es muy consistente ya que como se puede notar, una persona de 68 años puede ganar menos que alguien de 20 años pero una persona con 67 años gana más que alguien de 20 años. Ahora se determinará la relevancia de `income_type`

In [144]:
income_type_median = credit.groupby('income_type').agg({'total_income': 'median'}).reset_index()
income_type_median

,income_type,total_income
0,business,27577.2720
1,civil servant,24071.6695
2,employee,22815.1035
3,entrepreneur,79866.1030
4,paternity / maternity leave,8612.6610
5,retiree,18962.3180
6,student,15712.2600
7,unemployed,21014.3605


Esta columna es la más relevante ya que muestran los datos organizados por la manera en que los consiguen.

Para la columna de `total_income` se realizará un agrupamiento de los datos por el `income_type` para así realizar `median` en cada uno de los diferentes tipos de ingresos los cuales reemplazarán cada uno de los valores ausentes dependiendo del tipo de ingresos que esa persona tiene.

In [145]:
income_type_replace = {row["income_type"]:row["total_income"] for i, row in income_type_median.iterrows()}
credit['total_income'] = credit['total_income'].fillna(credit["income_type"].map(income_type_replace))
credit[credit['days_employed'].isnull()].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,18962.3180,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,24071.6695,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,18962.3180,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,24071.6695,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,18962.3180,to have a wedding


Se comprobará que el número total de valores en `total_income` coincida con el número de valores en otras columnas.

In [146]:
credit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21454 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21454 non-null  int64  
 3   education         21454 non-null  object 
 4   education_id      21454 non-null  int64  
 5   family_status     21454 non-null  object 
 6   family_status_id  21454 non-null  int64  
 7   gender            21454 non-null  object 
 8   income_type       21454 non-null  object 
 9   debt              21454 non-null  int64  
 10  total_income      21454 non-null  float64
 11  purpose           21454 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


###  Restaurar valores en `days_employed` <a id='restore_days_employed'></a>

Para restaurar los valores ausentes en `days_employed`, se tomarán en cuenta los datos de la columna `income_type` al igual que en el caso anterior ya que son los datos más relevantes para este estudio y reflejan un resultado muy útil.

In [147]:
days_employed_mean = credit.groupby('income_type').agg({'days_employed': 'mean'}).reset_index()
days_employed_mean

,income_type,days_employed
0,business,2111.524398
1,civil servant,3399.896902
2,employee,2326.499216
3,entrepreneur,520.848083
4,paternity / maternity leave,3296.759962
5,retiree,365003.491245
6,student,578.751554
7,unemployed,366413.652744


In [148]:
days_employed_median = credit.groupby('income_type').agg({'days_employed': 'median'}).reset_index()
days_employed_median

,income_type,days_employed
0,business,1547.382223
1,civil servant,2689.368353
2,employee,1574.202821
3,entrepreneur,520.848083
4,paternity / maternity leave,3296.759962
5,retiree,365213.306266
6,student,578.751554
7,unemployed,366413.652744


Se utilizarán las medias debido a que no hay valores fuera de lo común y representan un valor más preciso.

In [149]:
days_employed_replace = {row["income_type"]:row["days_employed"] for i, row in days_employed_mean.iterrows()}
credit['days_employed'] = credit['days_employed'].fillna(credit["income_type"].map(days_employed_replace))
credit[credit['days_employed'].isnull()].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


In [150]:
credit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21454 non-null  int64  
 1   days_employed     21454 non-null  float64
 2   dob_years         21454 non-null  int64  
 3   education         21454 non-null  object 
 4   education_id      21454 non-null  int64  
 5   family_status     21454 non-null  object 
 6   family_status_id  21454 non-null  int64  
 7   gender            21454 non-null  object 
 8   income_type       21454 non-null  object 
 9   debt              21454 non-null  int64  
 10  total_income      21454 non-null  float64
 11  purpose           21454 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Clasificación de datos <a id='data_classification'></a>

Primeramente se requiere analizar aquellas columnas que cuenten con ID para observar y determinar si será posible clasificar las demás columnas en base a IDs.

Empezando con `familiy_status` y `family_status_id`:

In [151]:
credit.drop_duplicates(['family_status_id', 'family_status'])[['family_status_id', 'family_status']]

,family_status_id,family_status
0,0,married
4,1,civil partnership
18,2,widow / widower
19,3,divorced
24,4,unmarried


No se encuentra una diferencia entre ambas columnas ya que son exactamente iguales y tienen los mismos valores representados de diferente manera. Se comprobará si ocurre lo mismo para `education` y `education_id`

In [152]:
credit.drop_duplicates(['education_id', 'education'])[['education_id', 'education']]

,education_id,education
0,0,bachelor's degree
1,1,secondary education
13,2,some college
31,3,primary education
2962,4,graduate degree


Como se puede observar, estas columnas también son idénticas y sólo cambia la manera en que se presentan los datos.

Para clasificar los datos se utilizarán sus IDs, es decir, se le asignará un ID a cada valor de las columnas para obtener una cantidad de IDs que sea mucho más clara y concisa. Así mismo, se crearán diccionarios para cada una de estas columnas con las claves siendo los IDs.

Se comenzará por agrupar la columna `purpose` ya que esta tiene el mismo significado escrito de maneras diferentes como se observa a continuación:

In [153]:
purpose_dicc = {0:['estate', 'real'], 1:'wedding', 2:'hous', 3:'property', 4:'car', 5:['educat', 'university'], 6:'N/A'}
family_dicc = {0:'lower_middle', 1:'middle', 2:'upper_middle', 3:'N/A'}

In [154]:
purpose_unique = credit['purpose'].unique()
purpose_unique

array(['purchase of the house', 'car purchase', 'supplementary education',
       'to have a wedding', 'housing transactions', 'education',
       'having a wedding', 'purchase of the house for my family',
       'buy real estate', 'buy commercial real estate',
       'buy residential real estate', 'construction of own property',
       'property', 'building a property', 'buying a second-hand car',
       'buying my own car', 'transactions with commercial real estate',
       'building a real estate', 'housing',
       'transactions with my real estate', 'cars', 'to become educated',
       'second-hand car purchase', 'getting an education', 'car',
       'wedding ceremony', 'to get a supplementary education',
       'purchase of my own house', 'real estate transactions',
       'getting higher education', 'to own a car', 'purchase of a car',
       'profile education', 'university education',
       'buying property for renting out', 'to buy a car',
       'housing renovation', 'going

In [155]:
def purpose_cleaning(purpose): 
    if 'estate' in purpose or 'real' in purpose:
        return 0
    elif 'wedding' in purpose:
        return 1
    elif 'hous' in purpose:
        return 2
    elif 'property' in purpose:
        return 3
    elif 'car' in purpose:
        return 4
    elif 'educat' in purpose or 'university' in purpose:
        return 5
    else:
        return 6

In [156]:
credit['purpose_id'] = credit['purpose'].apply(purpose_cleaning)
credit['purpose_id'].value_counts()

0    4464
4    4306
5    4013
2    3809
3    2538
1    2324
Name: purpose_id, dtype: int64

Una vez completado este paso, es posible continuar con las demás columnas que tendrán un beneficio de limpieza y claridad al realizar este mismo ejercicio en ellas. Las columnas en las cuales se realizará un reemplazo por IDs en lugar de su valor acutal (además de `purpose`) son `income_type`, `gender` y `total_income`.

Pra las columnas `education` y `family_status` simplemente se reemplazará una columna por la otra asegurando que sean los mismos valores.

Se eligieron estas columnas ya que tienen una cantidad de valores lo bastantemente bajas para justificar este cambio y que sean más fácil de leer los datos presentados.

In [157]:
lower_middle = credit['total_income'].quantile(q = 0.25)
middle = credit['total_income'].quantile(q = 0.5)
upper_middle = credit['total_income'].quantile(q = 0.75)

def total_income_id(income):
    if income <= lower_middle:
        return 0
    elif income <= middle:
        return 1
    elif income <= upper_middle:
        return 2
    else:
        return 3

In [158]:
credit['total_income_id'] = credit['total_income'].apply(total_income_id)
credit['total_income_id'].value_counts()

1    5479
3    5364
0    5364
2    5247
Name: total_income_id, dtype: int64

In [159]:
gender_id = {'M':0, 'F':1}
income_type_id = {income_type:i for i, income_type in enumerate(credit['income_type'].unique())}
education_id = {row['education']:row['education_id'] for i, row in credit.drop_duplicates(['education', 'education_id'])[['education', 'education_id']].iterrows()}
family_status_id = {row['family_status']:row['family_status_id'] for i, row in credit.drop_duplicates(['family_status', 'family_status_id'])[['family_status', 'family_status_id']].iterrows()}
print(gender_id)
print(income_type_id)
print(education_id)
family_status_id

{'M': 0, 'F': 1}
{'employee': 0, 'retiree': 1, 'business': 2, 'civil servant': 3, 'unemployed': 4, 'entrepreneur': 5, 'student': 6, 'paternity / maternity leave': 7}
{"bachelor's degree": 0, 'secondary education': 1, 'some college': 2, 'primary education': 3, 'graduate degree': 4}


{'married': 0,
 'civil partnership': 1,
 'widow / widower': 2,
 'divorced': 3,
 'unmarried': 4}

In [160]:
def data_processing():
    global credit
    credit['income_type_id'] = credit['income_type'].map(income_type_id)
    credit['gender_id'] = credit['gender'].map(gender_id)
    credit['gender_id'] = credit['gender_id'].astype(int)

data_processing()

Se procede a eliminar las columnas con los valores obsoletos

In [161]:
def columns_to_delete():
    global credit
    columns = ['gender', 'purpose', 'education', 'income_type', 'family_status', 'total_income']
    credit = credit.drop([col for col in columns if col in credit.columns], axis=1)
columns_to_delete()
credit.head(15)

,children,days_employed,dob_years,education_id,family_status_id,debt,purpose_id,total_income_id,income_type_id,gender_id
0,1,8437.673028,42,0,0,0,2,3,0,1
1,1,4024.803754,36,1,0,0,4,1,0,1
2,0,5623.422610,33,1,0,0,2,2,0,0
3,3,4124.747207,32,1,0,0,5,3,0,0
4,0,340266.072047,53,1,1,0,1,2,1,1
...,...,...,...,...,...,...,...,...,...,...
10,2,4171.483647,36,0,0,0,0,1,2,0
11,0,792.701887,40,1,0,0,0,0,0,1
12,0,365003.491245,65,1,1,0,1,1,1,0
13,0,1846.641941,54,2,0,0,4,1,0,1


In [162]:
credit[['income_type_id', 'education_id', 'family_status_id', 'gender_id', 'total_income_id', 'purpose_id']].apply(pd.Series.value_counts)

,income_type_id,education_id,family_status_id,gender_id,total_income_id,purpose_id
0,11084,5250.0,12339.0,7279.0,5364.0,4464.0
1,3829,15172.0,4151.0,14175.0,5479.0,2324.0
2,5078,744.0,959.0,NaN,5247.0,3809.0
3,1457,282.0,1195.0,NaN,5364.0,2538.0
4,2,6.0,2810.0,NaN,NaN,4306.0
5,2,NaN,NaN,NaN,NaN,4013.0
6,1,NaN,NaN,NaN,NaN,NaN
7,1,NaN,NaN,NaN,NaN,NaN


In [163]:
credit[['income_type_id', 'education_id', 'family_status_id', 'gender_id', 'total_income_id', 'purpose_id']].apply(pd.Series.value_counts, normalize = True)


,income_type_id,education_id,family_status_id,gender_id,total_income_id,purpose_id
0,0.516640,0.244710,0.575138,0.339284,0.250023,0.208073
1,0.178475,0.707187,0.193484,0.660716,0.255384,0.108325
2,0.236692,0.034679,0.044700,NaN,0.244570,0.177543
3,0.067913,0.013144,0.055701,NaN,0.250023,0.118300
4,0.000093,0.000280,0.130978,NaN,NaN,0.200708
5,0.000093,NaN,NaN,NaN,NaN,0.187051
6,0.000047,NaN,NaN,NaN,NaN,NaN
7,0.000047,NaN,NaN,NaN,NaN,NaN


## Comprobación de las hipótesis <a id='hypotheses'></a>


### ¿Existe una correlación entre tener hijos y pagar a tiempo? <a id='kids'></a>

In [164]:
print(credit['children'].value_counts())
for have_children in credit['children'].unique():
    number_children = credit['children'][(credit['children'] == have_children) & (credit['debt'] == 0)].count()
    data = credit['children'][credit['children'] == have_children].count()
    children_percent = round(number_children/ data * 100, 2)
    print(children_percent, have_children, number_children)
credit['children'].corr(credit['debt'], method = 'spearman')

0    14091
1     4855
2     2128
3      330
4       41
5        9
Name: children, dtype: int64
90.83 1 4410
92.46 0 13028
91.82 3 303
90.51 2 1926
90.24 4 37
100.0 5 9


0.028430810965430202

Como se puede observar, hay una diferencia mínima entre las personas con hijos que pagan a tiempo y las que no tienen hijos que pagan a tiempo ya que, del total de personas que no tienen hijos (14,091), el 92.46% (13,028) de ellos pagan a tiempo su préstamo mientras que el porcentaje más bajo de las peronsas con hijos se muestra en las familias con 4 hijos (90.24%), con una diferencia de menos del 2%. Así mismo, se nota una correlación muy baja entre ambas columnas.

A continuación se mostrará una `pivot_table` para mostrar los datos aún más claros.

In [165]:
pivot_table_children = credit.pivot_table(index = ['children'], values = ['debt'], aggfunc = ['sum','count'])
pivot_table_children['ratio'] = round(pivot_table_children[('sum', 'debt')] / pivot_table_children[('count', 'debt')] *100, 3)
pivot_table_children

,sum,count,ratio
,debt,debt,
children,,,
0,1063,14091,7.544
1,445,4855,9.166
2,202,2128,9.492
3,27,330,8.182
4,4,41,9.756
5,0,9,0.000


### ¿Existe una correlación entre la situación familiar y el pago a tiempo? <a id='family'></a>

In [166]:
print(credit['family_status_id'].value_counts())
print()
for fam_status in credit['family_status_id'].unique():
    fam_count = credit['family_status_id'][(credit['family_status_id'] == fam_status) & (credit['debt'] == 0)].count()
    data = credit['family_status_id'][credit['family_status_id'] == fam_status].count()
    fam_percent = round(fam_count / data * 100, 2)
    print(fam_percent, fam_status, fam_count)

0    12339
1     4151
4     2810
3     1195
2      959
Name: family_status_id, dtype: int64

92.45 0 11408
90.65 1 3763
93.43 2 896
92.89 3 1110
90.25 4 2536


De igual manera, estos resultados muestran un cumplimiento de pago muy alto, teniendo en cuenta que los cinco grupos están por encima del 90% de cumplimiento del pago a tiempo

Considerando que el significado de cada grupo es:

- `0` es 'married'
- `1` es 'civil partnership'
- `2` es 'widow/widower'
- `3` es 'divorced'
- `4` es 'unmarried'

Se mostrará a continuación una tabla dinámica para accesar a los valores de interés de manera clara y ordenada.

In [167]:
pivot_table_family_status_id = credit.pivot_table(index = ['family_status_id'], values = ['debt'], aggfunc = ['sum','count'])
pivot_table_family_status_id['ratio'] = round(pivot_table_family_status_id[('sum', 'debt')] / pivot_table_family_status_id[('count', 'debt')] *100, 3)
pivot_table_family_status_id

,sum,count,ratio
,debt,debt,
family_status_id,,,
0,931,12339,7.545
1,388,4151,9.347
2,63,959,6.569
3,85,1195,7.113
4,274,2810,9.751


### ¿Existe una correlación entre el nivel de ingresos y el pago a tiempo? <a id='income'></a>

In [168]:
print(credit['total_income_id'].value_counts())
print()
for total_income_type in credit['total_income_id'].unique():
    income_total = credit['total_income_id'][(credit['total_income_id'] == total_income_type) & (credit['debt'] == 0)].count()
    data = credit['total_income_id'][credit['total_income_id'] == total_income_type].count()
    total_income_percent = round(income_total / data * 100, 2)
    print(total_income_percent, total_income_type, income_total)
credit['total_income_id'].corr(credit['debt'], method = 'spearman')

1    5479
3    5364
0    5364
2    5247
Name: total_income_id, dtype: int64

92.86 3 4981
91.18 1 4996
91.46 2 4799
92.04 0 4937


-0.01121101352898981

Con estos resultados se puede llegar a la conclución de que de manera similar a los casos anteriores, no se encuentra una gran diferencia dentro de los grupos. La correlación es negativa lo cual significa que están un poco asociadas pero de manera inversa.

Teniendo en cuenta que:

- `0` es 'lower middle'
- `1` es 'middle'
- `2` es 'upper middle'
- `3` es 'lower high'

En la tabla dinámica se muestran los resultados.

In [169]:
pivot_table_total_income= credit.pivot_table(index = ['total_income_id'], values = ['debt'], aggfunc = ['sum','count'])
pivot_table_total_income['ratio'] = round(pivot_table_total_income[('sum', 'debt')] / pivot_table_total_income[('count', 'debt')] *100, 3)
pivot_table_total_income

,sum,count,ratio
,debt,debt,
total_income_id,,,
0,427,5364,7.960
1,483,5479,8.815
2,448,5247,8.538
3,383,5364,7.140


### ¿Cómo afecta el propósito del crédito a la tasa de incumplimiento? <a id='purpose'></a>

In [170]:
print(credit['purpose_id'].value_counts())
print()
for purpose_id_1 in credit['purpose_id'].unique():
    purpose_total = credit['purpose_id'][(credit['purpose_id'] == purpose_id_1) & (credit['debt'] == 0)].count()
    data = credit['purpose_id'][credit['purpose_id'] == purpose_id_1].count()
    purpose_percent = round(purpose_total / data * 100, 2)
    print(purpose_percent, purpose_id_1, purpose_total)

0    4464
4    4306
5    4013
2    3809
3    2538
1    2324
Name: purpose_id, dtype: int64

93.28 2 3553
90.64 4 3903
90.78 5 3643
92.0 1 2138
92.47 0 4128
92.51 3 2348


Este caso es similar a los anteriores donde no se observa una gran diferencia entre los grupos y todos tienen un porcentaje muy alto en pago a tiempo. Sin embargo, los datos muestran que las personas que tienen el propósito de comprar un carro y los que necesitan apoyo financiero para estudiar, son los que menos porcentaje tienen, esto se puede deber a que es más probable que personas más jóvenes pidan estos préstamos para estos propósitos.

Recordando que:

- `0` es 'real' o 'estate'
- `1` es 'wedding'
- `2` es 'house'
- `3` es 'property'
- `4` es 'car'
- `5` es 'education' o 'university'
- `6` es 'other'

La tabla dinámica de estos datos será mostrada a continuación:

In [171]:
pivot_table_purpose = credit.pivot_table(index = ['purpose_id'], values = ['debt'], aggfunc = ['sum','count'])
pivot_table_purpose['ratio'] = round(pivot_table_purpose[('sum', 'debt')] / pivot_table_purpose[('count', 'debt')] *100, 3)
pivot_table_purpose

,sum,count,ratio
,debt,debt,
purpose_id,,,
0,336,4464,7.527
1,186,2324,8.003
2,256,3809,6.721
3,190,2538,7.486
4,403,4306,9.359
5,370,4013,9.220


# Conclusión general <a id='end'></a>

Después de analizar el conjunto de datos, se puede notar que:

- Todos los clientes en `family_status_id`, `children`, `total_income_id` y `purpose_id` pagan sus préstamos a tiempo en más del 90% de las ocaciones.
- Las familias con 5 hijos pagan el 100% de las veces a tiempo.
- Las personas que buscan el préstamo para comprar un carro, con ingresos medios, n ose casaron  o no están casadas y las personas con 4 hijos, son las que más bajo porcentaje de pagos a tiempo tienen.
- La correlación más alta se encuentra en la cantidad de hijos y el pago a tiempo con apenas 0.284
